# Transfer Learning for Glacier Mass Balance Prediction

This notebook demonstrates **transfer learning** between glacier mass balance models trained on different regions. Specifically, we:

1. **Load a pre-trained neural network** trained on Swiss glacier data
2. **Fine-tune it on Icelandic glacier data** using various strategies
3. **Evaluate performance** on unseen Icelandic glaciers

## Key Features
- **Progressive layer unfreezing** for gradual adaptation
- **Multiple train/test split strategies** (50%, North/South, 5-10%)
- **Comprehensive evaluation metrics** and visualizations
- **Model checkpointing** at specific epochs for analysis

---

## Prerequisites
- Pre-trained model on all Swiss data from ../regions/Switzerland/3.2.2 Train-ML-model-NN.ipynb e.g. `nn_model_2025-07-14_CH_flexible.pt`
- Icelandic glacier dataset from ../regions/Iceland_mb/1.1. Iceland-prepro.ipynb
- ERA5 climate data of Iceland from ../regions/Iceland_mb/1.2. ERA5Land-prepro.ipynb

---

In [1]:
# Add root of repo to import MBM
import sys, os
sys.path.append(os.path.join(os.getcwd(), '../../'))

# Core libraries
import pandas as pd
import warnings
from tqdm.notebook import tqdm
import re
#import pickle # for displaying saved model parameters etc.
from datetime import datetime
from collections import defaultdict
import logging

# Visualization
import matplotlib.pyplot as plt
import seaborn as sns
from cmcrameri import cm

# Scientific computing
import xarray as xr
import torch
import torch.nn as nn
from torch.utils.data import Dataset
from torch.optim.lr_scheduler import ReduceLROnPlateau

# Skorch (scikit-learn compatible PyTorch)
from skorch.helper import SliceDataset
from skorch.callbacks import EarlyStopping, LRScheduler, Checkpoint

# MassBalanceMachine (custom package)
import massbalancemachine as mbm

# Local helper modules

from scripts.helpers import *
from scripts.iceland_preprocess import *
from scripts.plots import *
from scripts.config_ICE import *
from scripts.nn_helpers import *
from scripts.xgb_helpers import *
from scripts.NN_networks import *

warnings.filterwarnings('ignore')
%load_ext autoreload
%autoreload 2

# Initialize logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(message)s')

cfg = mbm.IcelandConfig(dataPath='/home/mburlet/scratch/data/DATA_MB/WGMS/Iceland/')

In [2]:
# Set random seeds for reproducibility
seed_all(cfg.seed)
free_up_cuda()

# Configure plotting style
path_style_sheet = 'scripts/example.mplstyle'
plt.style.use(path_style_sheet)

# Define climate features from ERA5 reanalysis
vois_climate = [
    't2m',     # 2-meter temperature
    'tp',      # Total precipitation  
    'slhf',    # Surface latent heat flux
    'sshf',    # Surface sensible heat flux
    'ssrd',    # Surface solar radiation downwards
    'fal',     # Forecast albedo
    'str',     # Surface thermal radiation
    'u10',     # 10-meter U wind component
    'v10'      # 10-meter V wind component
]

# Define topographical features from OGGM
vois_topographical = [
    "aspect",                    # Terrain aspect (OGGM)
    "slope",                     # Terrain slope (OGGM)
    "hugonnet_dhdt",            # Ice thickness change (OGGM)
    "consensus_ice_thickness",   # Ice thickness consensus (OGGM)
    "millan_v",                 # Ice velocity (OGGM)
]

In [3]:
# Ensure reproducibility across runs
seed_all(cfg.seed)

# Check for CUDA availability and configure accordingly
if torch.cuda.is_available():
    print("CUDA is available")
    free_up_cuda()

    # # Try to limit CPU usage of random search
    # torch.set_num_threads(2)  # or 1
    # os.environ["OMP_NUM_THREADS"] = "1"
    # os.environ["MKL_NUM_THREADS"] = "1"
else:
    print("CUDA is NOT available")


CUDA is available


## Data Loading & Preprocessing

### Create Icelandic Glacier Dataset
We start with point mass balance measurements and transform them to monthly format with ERA5 climate data.

In [4]:
# Load Icleandic glacier dataset with topographical features
data_wgms = pd.read_csv(cfg.dataPath + path_PMB_WGMS_csv + 'ICE_dataset_all_oggm_with_hugonnetdhdt.csv')

# Remove entries with missing hugonnet_dhdt data
data_wgms = data_wgms.dropna(subset=data_wgms.columns.drop('DATA_MODIFICATION'))

print('Number of glaciers:', len(data_wgms['GLACIER'].unique()))
print('Number of winter, summer and annual samples:', len(data_wgms[data_wgms.PERIOD == 'annual']) + len(data_wgms[data_wgms.PERIOD == 'winter']) + len(data_wgms[data_wgms.PERIOD == 'summer']))
print('Number of annual samples:',
      len(data_wgms[data_wgms.PERIOD == 'annual']))
print('Number of winter samples:',
      len(data_wgms[data_wgms.PERIOD == 'winter']))
print('Number of summer samples:',
      len(data_wgms[data_wgms.PERIOD == 'summer']))

data_wgms.columns


Number of glaciers: 47
Number of winter, summer and annual samples: 5930
Number of annual samples: 2883
Number of winter samples: 3047
Number of summer samples: 0


Index(['POINT_ELEVATION', 'POINT_LAT', 'POINT_LON', 'DATA_MODIFICATION',
       'FROM_DATE', 'TO_DATE', 'POINT_BALANCE', 'PERIOD', 'YEAR', 'RGIId',
       'aspect', 'slope', 'topo', 'hugonnet_dhdt', 'consensus_ice_thickness',
       'millan_v', 'GLACIER', 'POINT_ID'],
      dtype='object')

In [5]:
data_ICE_test = data_wgms.copy()

# Transform data to monthly format (run or load data):
paths = {
    'csv_path': cfg.dataPath + path_PMB_WGMS_csv,
    'era5_climate_data': cfg.dataPath + path_ERA5_raw + 'era5_monthly_averaged_data_ICECH.nc',
    'geopotential_data': cfg.dataPath + path_ERA5_raw + 'era5_geopotential_pressure_ICECH.nc'
}

# Transform point measurements to monthly format and merge with ERA5 climate data
# Set RUN=True to reprocess data, False to load existing preprocessed file
RUN = False
dataloader_gl = process_or_load_data(run_flag=RUN,
                                     df=data_ICE_test,
                                     paths=paths,
                                     cfg=cfg,
                                     vois_climate=vois_climate,
                                     vois_topographical=vois_topographical,
                                     output_file= 'ICE_dataset_monthly_full_with_hugonnetdhdt.csv')
data_monthly = dataloader_gl.data

display(data_monthly.head(2))

2025-09-09 17:55:00,157 - INFO - Loaded preprocessed data.
2025-09-09 17:55:00,158 - INFO - Number of monthly rows: 56209
2025-09-09 17:55:00,166 - INFO - Number of annual samples: 34915
2025-09-09 17:55:00,171 - INFO - Number of winter samples: 21294


,YEAR,POINT_LON,POINT_LAT,POINT_BALANCE,ALTITUDE_CLIMATE,ELEVATION_DIFFERENCE,POINT_ELEVATION,RGIId,POINT_ID,ID,...,millan_v,t2m,tp,slhf,sshf,ssrd,fal,str,u10,v10
0,1989.0,-18.595688,64.790063,0.45,1094.738918,-45.138918,1049.6,RGI60-06.00234,Thjorsarjoekull (Hofsjoekull E)_1989.0_annual_...,0,...,25.446609,-4.787933,0.003747,198886.0,590722.0,3990646.0,0.84564,-1779436.0,-0.590102,0.603381
1,1989.0,-18.595688,64.790063,0.45,1094.738918,-45.138918,1049.6,RGI60-06.00234,Thjorsarjoekull (Hofsjoekull E)_1989.0_annual_...,0,...,25.446609,-5.262054,0.003766,306376.0,576896.0,883468.0,0.84690,-1458576.0,0.129348,2.214950


In [6]:
display(data_monthly['GLACIER'].value_counts())
display(data_monthly.shape)

display()

GLACIER
RGI60-06.00238                                8230
Bruarjoekull                                  6814
Skeidararjoekull                              5008
Thjorsarjoekull (Hofsjoekull E)               3962
Blagnipujoekull (Hofsjoekull SW)              3317
Hagafellsjoekull West                         3207
Breidamerkurjoekull                           2953
Koeldukvislarjoekull                          2897
Langjoekull Ice Cap                           2647
Dyngjujoekull                                 2305
Eyjabakkajoekull                              2110
Tungnaarjoekull                               1430
Hagafellsjoekull East (Langjoekull S Dome)    1050
RGI60-06.00292                                1001
RGI60-06.00302                                 973
RGI60-06.00478                                 957
RGI60-06.00294                                 946
RGI60-06.00311                                 857
RGI60-06.00465                                 741
Mulajoekull            

(56209, 28)

## Train/Test Split Strategies

Implement three different strategies for splitting the Norway data to test various transfer learning scenarios:

### Strategy 1: 50% Random Split
**Data**: About 50 % of the available data is used as fine-tuning set, which have been chosen from to be representative of the 4 available ice caps.

**Use case**: Balanced representation with good data availability for transfer learning adaptation.

In [7]:
# TRANSFER LEARNING SETUP 50%
# Fine-tuning glaciers
#train_glaciers = ['Bruarjoekull', 'Skeidararjoekull', 'Koeldukvislarjoekull', 'Slettjoekull West', 'RGI60-06.00238', 'Hagafellsjoekull West']

train_glaciers = ['RGI60-06.00228',
 'RGI60-06.00445',
 'RGI60-06.00480',
 'RGI60-06.00306',
 'Hagafellsjoekull East (Langjoekull S Dome)',
 'Tungnaarjoekull',
 'Thjorsarjoekull (Hofsjoekull E)',
 'RGI60-06.00413',
 'RGI60-06.00359',
 'Slettjoekull West',
 'Hagafellsjoekull West',
 'Skeidararjoekull',
 'Blagnipujoekull (Hofsjoekull SW)',
 'Bruarjoekull',
 'RGI60-06.00422',
 'RGI60-06.00301',
 'Mulajoekull',
 'RGI60-06.00411',
 'RGI60-06.00294',
 'RGI60-06.00305',
 'RGI60-06.00466']

# Test glaciers (all remaining Iceland glaciers)
all_iceland_glaciers = list(data_wgms['GLACIER'].unique())
test_glaciers = [g for g in all_iceland_glaciers if g not in train_glaciers]

print(f"Fine-tuning glaciers ({len(train_glaciers)}): {train_glaciers}")
print(f"Test glaciers ({len(test_glaciers)}): {test_glaciers}")

# Ensure all glaciers exist in the dataset
existing_glaciers = set(dataloader_gl.data.GLACIER.unique())
missing_fine_tune = [g for g in train_glaciers if g not in existing_glaciers]
missing_test = [g for g in test_glaciers if g not in existing_glaciers]

if missing_fine_tune:
    print(f"Warning: Fine-tuning glaciers not in dataset: {missing_fine_tune}")
if missing_test:
    print(f"Warning: Test glaciers not in dataset: {missing_test}")


# Use helper function from XGBoost to create test/train set. CV splits are not used here.
splits, test_set, train_set = get_CV_splits(dataloader_gl,
                                            test_split_on='GLACIER',
                                            test_splits=test_glaciers,
                                            random_state=cfg.seed)
    
print('Train glaciers: ({}) {}'.format(len(train_set['splits_vals']),
                                      train_set['splits_vals']))
print('Test glaciers: ({}) {}'.format(len(test_set['splits_vals']),
                                      test_set['splits_vals']))
display('length train set', len(train_set['df_X']))
display('length test set', len(test_set['df_X']))


Fine-tuning glaciers (21): ['RGI60-06.00228', 'RGI60-06.00445', 'RGI60-06.00480', 'RGI60-06.00306', 'Hagafellsjoekull East (Langjoekull S Dome)', 'Tungnaarjoekull', 'Thjorsarjoekull (Hofsjoekull E)', 'RGI60-06.00413', 'RGI60-06.00359', 'Slettjoekull West', 'Hagafellsjoekull West', 'Skeidararjoekull', 'Blagnipujoekull (Hofsjoekull SW)', 'Bruarjoekull', 'RGI60-06.00422', 'RGI60-06.00301', 'Mulajoekull', 'RGI60-06.00411', 'RGI60-06.00294', 'RGI60-06.00305', 'RGI60-06.00466']
Test glaciers (26): ['Breidamerkurjoekull', 'Dyngjujoekull', 'RGI60-06.00328', 'Eyjabakkajoekull', 'RGI60-06.00303', 'Langjoekull Ice Cap', 'Koeldukvislarjoekull', 'Oeldufellsjoekull', 'RGI60-06.00302', 'RGI60-06.00340', 'RGI60-06.00238', 'RGI60-06.00320', 'RGI60-06.00342', 'RGI60-06.00465', 'RGI60-06.00292', 'RGI60-06.00232', 'RGI60-06.00478', 'RGI60-06.00311', 'RGI60-06.00350', 'RGI60-06.00476', 'RGI60-06.00409', 'RGI60-06.00349', 'RGI60-06.00425', 'RGI60-06.00479', 'RGI60-06.00296', 'RGI60-06.00474']
Train glaciers

'length train set'

28791

'length test set'

27418

### Strategy 2: East/West Geographic Split

**Data:** Western glaciers are used as fine-tuning set, eastern as test set. Split is approx. 50% of data.

**Use case**: Test geographic generalization across longitudinal gradients.

In [6]:
# Get glacier latitudes
glacier_lat = data_wgms.groupby('GLACIER')['POINT_LON'].first()

# Use the median latitude as the split
lon_threshold = -18.25

east_glaciers = glacier_lat[glacier_lat >= lon_threshold].index.tolist()
west_glaciers = glacier_lat[glacier_lat < lon_threshold].index.tolist()

print(f"East glaciers ({len(east_glaciers)}): {east_glaciers}")
print(f"West glaciers ({len(west_glaciers)}): {west_glaciers}")

East glaciers (20): ['Breidamerkurjoekull', 'Bruarjoekull', 'Dyngjujoekull', 'Eyjabakkajoekull', 'Koeldukvislarjoekull', 'RGI60-06.00409', 'RGI60-06.00411', 'RGI60-06.00413', 'RGI60-06.00422', 'RGI60-06.00425', 'RGI60-06.00445', 'RGI60-06.00465', 'RGI60-06.00466', 'RGI60-06.00474', 'RGI60-06.00476', 'RGI60-06.00478', 'RGI60-06.00479', 'RGI60-06.00480', 'Skeidararjoekull', 'Tungnaarjoekull']
West glaciers (27): ['Blagnipujoekull (Hofsjoekull SW)', 'Hagafellsjoekull East (Langjoekull S Dome)', 'Hagafellsjoekull West', 'Langjoekull Ice Cap', 'Mulajoekull', 'Oeldufellsjoekull', 'RGI60-06.00228', 'RGI60-06.00232', 'RGI60-06.00238', 'RGI60-06.00292', 'RGI60-06.00294', 'RGI60-06.00296', 'RGI60-06.00301', 'RGI60-06.00302', 'RGI60-06.00303', 'RGI60-06.00305', 'RGI60-06.00306', 'RGI60-06.00311', 'RGI60-06.00320', 'RGI60-06.00328', 'RGI60-06.00340', 'RGI60-06.00342', 'RGI60-06.00349', 'RGI60-06.00350', 'RGI60-06.00359', 'Slettjoekull West', 'Thjorsarjoekull (Hofsjoekull E)']


In [ ]:
train_glaciers = east_glaciers

# Test glaciers (all remaining Iceland glaciers)
all_iceland_glaciers = list(data_wgms['GLACIER'].unique())
test_glaciers = [g for g in all_iceland_glaciers if g not in train_glaciers]

print(f"Fine-tuning glaciers ({len(train_glaciers)}): {train_glaciers}")
print(f"Test glaciers ({len(test_glaciers)}): {test_glaciers}")

# Ensure all glaciers exist in the dataset
existing_glaciers = set(dataloader_gl.data.GLACIER.unique())
missing_fine_tune = [g for g in train_glaciers if g not in existing_glaciers]
missing_test = [g for g in test_glaciers if g not in existing_glaciers]

if missing_fine_tune:
    print(f"Warning: Fine-tuning glaciers not in dataset: {missing_fine_tune}")
if missing_test:
    print(f"Warning: Test glaciers not in dataset: {missing_test}")


# Use helper function from XGBoost to create test/train set. CV splits are not used here.
splits, test_set, train_set = get_CV_splits(dataloader_gl,
                                            test_split_on='GLACIER',
                                            test_splits=test_glaciers,
                                            random_state=cfg.seed)
    
print('Train glaciers: ({}) {}'.format(len(train_set['splits_vals']),
                                      train_set['splits_vals']))
print('Test glaciers: ({}) {}'.format(len(test_set['splits_vals']),
                                      test_set['splits_vals']))

display('length train set', len(train_set['df_X']))
display('length test set', len(test_set['df_X']))

Fine-tuning glaciers (20): ['Breidamerkurjoekull', 'Bruarjoekull', 'Dyngjujoekull', 'Eyjabakkajoekull', 'Koeldukvislarjoekull', 'RGI60-06.00409', 'RGI60-06.00411', 'RGI60-06.00413', 'RGI60-06.00422', 'RGI60-06.00425', 'RGI60-06.00445', 'RGI60-06.00465', 'RGI60-06.00466', 'RGI60-06.00474', 'RGI60-06.00476', 'RGI60-06.00478', 'RGI60-06.00479', 'RGI60-06.00480', 'Skeidararjoekull', 'Tungnaarjoekull']
Test glaciers (27): ['Thjorsarjoekull (Hofsjoekull E)', 'RGI60-06.00328', 'Hagafellsjoekull West', 'RGI60-06.00303', 'Langjoekull Ice Cap', 'Oeldufellsjoekull', 'Slettjoekull West', 'RGI60-06.00302', 'RGI60-06.00359', 'RGI60-06.00340', 'Blagnipujoekull (Hofsjoekull SW)', 'RGI60-06.00238', 'RGI60-06.00320', 'Hagafellsjoekull East (Langjoekull S Dome)', 'RGI60-06.00342', 'RGI60-06.00294', 'RGI60-06.00292', 'RGI60-06.00232', 'Mulajoekull', 'RGI60-06.00301', 'RGI60-06.00311', 'RGI60-06.00350', 'RGI60-06.00228', 'RGI60-06.00349', 'RGI60-06.00305', 'RGI60-06.00306', 'RGI60-06.00296']
Train glaciers

'length train set'

26777

'length test set'

29432

### Strategy 3: Limited Data Split (5-10% for fine-tuning)

**Data**: About 5-10 % of the available data is used as fine-tuning set, consisting of 5 glaciers, which have been chosen from to be representative of the 4 available ice caps.

**Use case**: Test performance with minimal fine-tuning data (~500 measurements) to simulate data-scarce scenarios.

In [7]:
# TRANSFER LEARNING SETUP: 5-10% Limited Data Strategy

train_glaciers = ['Mulajoekull' ,'Slettjoekull West', 'Hagafellsjoekull East (Langjoekull S Dome)', 'Tungnaarjoekull', 'RGI60-06.00478']

# Test glaciers (all remaining Iceland glaciers)
all_iceland_glaciers = list(data_wgms['GLACIER'].unique())
test_glaciers = [g for g in all_iceland_glaciers if g not in train_glaciers]

print(f"Fine-tuning glaciers ({len(train_glaciers)}): {train_glaciers}")
print(f"Test glaciers ({len(test_glaciers)}): {test_glaciers}")

# Ensure all glaciers exist in the dataset
existing_glaciers = set(dataloader_gl.data.GLACIER.unique())
missing_fine_tune = [g for g in train_glaciers if g not in existing_glaciers]
missing_test = [g for g in test_glaciers if g not in existing_glaciers]

if missing_fine_tune:
    print(f"Warning: Fine-tuning glaciers not in dataset: {missing_fine_tune}")
if missing_test:
    print(f"Warning: Test glaciers not in dataset: {missing_test}")


# Use helper function from XGBoost to create test/train set. CV splits are not used here.
splits, test_set, train_set = get_CV_splits(dataloader_gl,
                                            test_split_on='GLACIER',
                                            test_splits=test_glaciers,
                                            random_state=cfg.seed)
    
print('Train glaciers: ({}) {}'.format(len(train_set['splits_vals']),
                                      train_set['splits_vals']))
print('Test glaciers: ({}) {}'.format(len(test_set['splits_vals']),
                                      test_set['splits_vals']))

display('length train set', len(train_set['df_X']))
display('length test set', len(test_set['df_X']))

Fine-tuning glaciers (5): ['Mulajoekull', 'Slettjoekull West', 'Hagafellsjoekull East (Langjoekull S Dome)', 'Tungnaarjoekull', 'RGI60-06.00478']
Test glaciers (42): ['Thjorsarjoekull (Hofsjoekull E)', 'Breidamerkurjoekull', 'Dyngjujoekull', 'RGI60-06.00328', 'Hagafellsjoekull West', 'Eyjabakkajoekull', 'RGI60-06.00303', 'Langjoekull Ice Cap', 'Koeldukvislarjoekull', 'Oeldufellsjoekull', 'Skeidararjoekull', 'RGI60-06.00466', 'RGI60-06.00411', 'RGI60-06.00302', 'RGI60-06.00359', 'RGI60-06.00340', 'Bruarjoekull', 'Blagnipujoekull (Hofsjoekull SW)', 'RGI60-06.00238', 'RGI60-06.00320', 'RGI60-06.00342', 'RGI60-06.00480', 'RGI60-06.00465', 'RGI60-06.00294', 'RGI60-06.00292', 'RGI60-06.00232', 'RGI60-06.00301', 'RGI60-06.00413', 'RGI60-06.00311', 'RGI60-06.00350', 'RGI60-06.00476', 'RGI60-06.00228', 'RGI60-06.00409', 'RGI60-06.00349', 'RGI60-06.00422', 'RGI60-06.00305', 'RGI60-06.00425', 'RGI60-06.00306', 'RGI60-06.00479', 'RGI60-06.00296', 'RGI60-06.00445', 'RGI60-06.00474']
Train glaciers:

'length train set'

4393

'length test set'

51816

## Validation Split Options

Use the same Option as was used for the Swiss model.

### Option 1: Random 80/20 Split
**Recommended for**: General model validation with balanced representation across all fine-tuning glaciers.

In [8]:
# Validation and train split:
data_train = train_set['df_X']
data_train['y'] = train_set['y']
dataloader = mbm.dataloader.DataLoader(cfg, data=data_train)

# Create random train/validation split
train_itr, val_itr = dataloader.set_train_test_split(test_size=0.2)

# Get all indices of the training and valdating dataset at once from the iterators. Once called, the iterators are empty.
train_indices, val_indices = list(train_itr), list(val_itr)

# Create training subset
df_X_train = data_train.iloc[train_indices]
y_train = df_X_train['POINT_BALANCE'].values

# Create validation subset
df_X_val = data_train.iloc[val_indices]
y_val = df_X_val['POINT_BALANCE'].values


print("Train data glacier distribution:", df_X_train['GLACIER'].value_counts().head())
print("Val data glacier distribution:", df_X_val['GLACIER'].value_counts().head())
print("Train data shape:", df_X_train.shape)
print("Val data shape:", df_X_val.shape)

Train data glacier distribution: GLACIER
Bruarjoekull                        5522
Skeidararjoekull                    3955
Thjorsarjoekull (Hofsjoekull E)     3027
Blagnipujoekull (Hofsjoekull SW)    2622
Hagafellsjoekull West               2621
Name: count, dtype: int64
Val data glacier distribution: GLACIER
Bruarjoekull                        1292
Skeidararjoekull                    1053
Thjorsarjoekull (Hofsjoekull E)      935
Blagnipujoekull (Hofsjoekull SW)     695
Hagafellsjoekull West                586
Name: count, dtype: int64
Train data shape: (23014, 29)
Val data shape: (5777, 29)


### Option 2: Glacier-wise Train/Val Split
**Recommended for**: Testing glacier-level generalization by validating on a completely unseen glacier during fine-tuning.

In [ ]:

data_train = train_set['df_X']
data_train['y'] = train_set['y']

val_glacier = ['Engabreen']
train_glaciers = [g for g in train_glaciers if g not in val_glacier]

# Create training subset (excluding validation glacier)
df_X_train = data_train[data_train['GLACIER'].isin(train_glaciers)].copy()
y_train = df_X_train['POINT_BALANCE'].values

# Create validation subset (only validation glacier)
df_X_val = data_train[data_train['GLACIER'].isin(val_glacier)].copy()
y_val = df_X_val['POINT_BALANCE'].values

print("Train data glacier distribution:", df_X_train['GLACIER'].value_counts().head())
print("Val data glacier distribution:", df_X_val['GLACIER'].value_counts().head())
print("Train data shape:", df_X_train.shape)
print("Val data shape:", df_X_val.shape)

Train data glacier distribution: GLACIER
Graafjellsbrea        2365
Breidablikkbrea       1593
Austre Memurubreen    1580
Rembesdalskaaka       1561
Tunsbergdalsbreen     1517
Name: count, dtype: int64
Val data glacier distribution: Series([], Name: count, dtype: int64)
Train data shape: (13698, 29)
Val data shape: (0, 29)


## Neural Network Configuration

### Feature Engineering and Dataset Preparation

In [ ]:
# Define complete feature set for model training
features_topo = [
    'ELEVATION_DIFFERENCE',
] + list(vois_topographical)

# Combine topographical and climate features
feature_columns = features_topo + list(vois_climate)

# Set features in config
cfg.setFeatures(feature_columns)

# Include all necessary columns (features + metadata)
all_columns = feature_columns + cfg.fieldsNotFeatures

df_X_train_subset = df_X_train[all_columns]
df_X_val_subset = df_X_val[all_columns]
df_X_test_subset = test_set['df_X'][all_columns]

print('Shape of training dataset:', df_X_train_subset.shape)
print('Shape of validation dataset:', df_X_val_subset.shape)
print('Shape of testing dataset:', df_X_test_subset.shape)
print('Running with features:', feature_columns)

# Sanity check: ensure targets match features
assert all(train_set['df_X'].POINT_BALANCE == train_set['y']), "Target mismatch detected!"
print('Feature-target alignment verified')

Shape of training dataset: (23014, 28)
Shape of validation dataset: (5777, 28)
Shape of testing dataset: (27418, 28)
Running with features: ['ELEVATION_DIFFERENCE', 'aspect', 'slope', 'hugonnet_dhdt', 'consensus_ice_thickness', 'millan_v', 't2m', 'tp', 'slhf', 'sshf', 'ssrd', 'fal', 'str', 'u10', 'v10']
Feature-target alignment verified


### Model Callbacks and Training Configuration
Set up training callbacks and configuration for optimal performance and monitoring.

In [10]:
# Early stopping to prevent overfitting
early_stop = EarlyStopping(
    monitor='valid_loss',    # Monitor validation loss
    patience=15,             # Stop after 15 epochs without improvement
    threshold=1e-4,          # Minimum change threshold
)

# Learning rate scheduler for adaptive training
lr_scheduler_cb = LRScheduler(
    policy=ReduceLROnPlateau,
    monitor='valid_loss',
    mode='min',
    factor=0.5,              # Reduce LR by half
    patience=5,              # Wait 5 epochs before reducing
    threshold=0.01,
    threshold_mode='rel',
    verbose=True
)

# Global variables for dataset management
dataset = dataset_val = None

def my_train_split(ds, y=None, **fit_params):
    """Custom train/validation split function for skorch."""
    return dataset, dataset_val

# Model configuration parameters
param_init = {'device': 'cpu'}
nInp = len(feature_columns)  # Number of input features

# Model checkpointing to save best model during training
checkpoint_cb = Checkpoint(
    monitor='valid_loss_best',
    f_params='best_model.pt',
    f_optimizer=None,        # Don't save optimizer state
    f_history=None,          # Don't save training history
    f_criterion=None,        # Don't save criterion state
    load_best=True,          # Load best model after training
)

# Custom callback to save models at specific epochs for analysis
save_best_epochs_cb = SaveBestAtEpochs([10, 15, 20, 30, 50, 100])

print('Callbacks and configuration ready!')

Callbacks and configuration ready!


### Dataset Creation
Datasets will be created in the training loop after loading the pre-trained Swiss model to ensure compatible preprocessing.

In [11]:
# Initialize dataset variables as None
features = features_val = None
metadata = metadata_val = None
dataset = dataset_val = None

print("Dataset creation deferred until Swiss model is loaded...")

Dataset creation deferred until Swiss model is loaded...


## Transfer Learning Execution

### Loading Pre-trained Swiss Model and Fine-tuning on Norwegian Data

### Method 1: Standard Fine-tuning with Selective Layer Freezing

After loading the model, all layers will be frozen by default, to unfreeze a layer you have to include it in "if name not in [...]" in Step 3.

 The SaveBestAtEpochs callback automatically saves the current best model at epochs [10, 15, 20, 30, 50, 100], which can then be evaluated in the Epoch-wise model evalution section. Comment out the callback if you don't want this feature. If you do and you continuously want to retrain models at different learning rates, you have to reexecute the "save_best_epochs_cb = SaveBestAtEpochs([10, 15, 20, 30, 50, 100])" cell.

In [20]:
TRAIN = True  # Set to True to actually train

if TRAIN:
    # STEP 1: Load the pre-trained Swiss model FIRST
    print("Loading pre-trained Swiss model...")
    model_filename = "nn_model_2025-07-14_CH_flexible.pt"
    
    # Define Swiss model architecture and parameters
    swiss_args = {
        'module': FlexibleNetwork,
        'nbFeatures': nInp,
        'module__input_dim': nInp,
        'module__dropout': 0.2,
        'module__hidden_layers': [128, 128, 64, 32],
        'module__use_batchnorm': True,
        'warm_start': True,             # CRITICAL: preserve pretrained weights
        'train_split': my_train_split,
        'batch_size': 128,
        'verbose': 1,
        'iterator_train__shuffle': True,
        'lr': 0.001,
        'max_epochs': 200,
        'optimizer': torch.optim.Adam,
        'optimizer__weight_decay': 1e-05,
        'callbacks': [
            ('early_stop', early_stop),
            ('lr_scheduler', lr_scheduler_cb),
            ('checkpoint', checkpoint_cb),
            #('save_best_at_epochs', save_best_epochs_cb)  # Save models at specific epochs
        ]
    }
    
    # Load the pre-trained model
    loaded_model = mbm.models.CustomNeuralNetRegressor.load_model(
        cfg, model_filename, **{**swiss_args, **param_init}
    )

    print("✓ Swiss model loaded successfully!")
    
    # STEP 2: Create datasets using the loaded Swiss model
    print("Creating datasets with Swiss model...")
    features, metadata = loaded_model._create_features_metadata(df_X_train_subset)
    features_val, metadata_val = loaded_model._create_features_metadata(df_X_val_subset)
    
    # Create global datasets
    dataset = mbm.data_processing.AggregatedDataset(cfg,
                                                    features=features,
                                                    metadata=metadata,
                                                    targets=y_train)
    dataset = mbm.data_processing.SliceDatasetBinding(SliceDataset(dataset, idx=0),
                                                      SliceDataset(dataset, idx=1))
    
    dataset_val = mbm.data_processing.AggregatedDataset(cfg,
                                                        features=features_val,
                                                        metadata=metadata_val,
                                                        targets=y_val)
    dataset_val = mbm.data_processing.SliceDatasetBinding(SliceDataset(dataset_val, idx=0), 
                                                          SliceDataset(dataset_val, idx=1))
    
    print("train:", dataset.X.shape, dataset.y.shape)
    print("validation:", dataset_val.X.shape, dataset_val.y.shape)


    # STEP 3: Apply selective layer freezing
    for name, param in loaded_model.module_.named_parameters():
        if name not in [#'model.0.weight', 'model.0.bias',
                        'model.1.weight', 'model.1.bias',
                        #'model.4.weight', 'model.4.bias',
                        'model.5.weight', 'model.5.bias',
                        #'model.8.weight', 'model.8.bias',
                        'model.9.weight', 'model.9.bias',
                        #'model.12.weight', 'model.12.bias',
                        'model.13.weight', 'model.13.bias',
                        #'model.16.weight', 'model.16.bias'
                        ]:
            param.requires_grad = False
    
    # STEP 4: Configure for fine-tuning
    print("Updating model for fine-tuning...")
    loaded_model = loaded_model.set_params(
        lr=0.05,
        max_epochs=200,
    )
    
    # STEP 5: Execute fine-tuning
    print("Starting fine-tuning...")
    loaded_model.fit(features, y_train)
    
    # STEP 6: Save fine-tuned model
    current_date = datetime.now().strftime("%Y-%m-%d")
    finetuned_model_filename = f"nn_model_finetuned_{current_date}"
    loaded_model.save_model(finetuned_model_filename)
    print(f"✓ Fine-tuned model saved as: {finetuned_model_filename}")

else:
    print("Training skipped (TRAIN=False)")


Loading pre-trained Swiss model...
✓ Swiss model loaded successfully!
Creating datasets with Swiss model...
train: (2424,) (2424,)
validation: (606,) (606,)
Updating model for fine-tuning...
Starting fine-tuning...
  epoch    train_loss    valid_loss    cp      lr     dur
-------  ------------  ------------  ----  ------  ------
      1        1.5736        1.4403     +  0.0500  1.2073
      2        1.2826        1.3215     +  0.0500  1.2307
      3        1.1408        1.2599     +  0.0500  1.2725
      4        1.0674        1.4864        0.0500  1.1808
      5        1.1052        1.4669        0.0500  1.1689
      6        1.2073        1.3130        0.0500  1.7006
      7        1.1675        1.1890     +  0.0500  1.5022
      8        1.0811        1.2145        0.0500  1.2930
      9        1.0546        1.4022        0.0500  1.3192
     10        1.0355        1.1983        0.0500  1.2233
     11        1.0572        1.4795        0.0500  1.3308
     12        1.0371        1.

### Method 2: Progressive Layer Unfreezing
This advanced approach gradually unfreezes layers during training for more controlled adaptation to the Icelandic data.

In [ ]:
TRAIN = True  # Set to True to actually train

if TRAIN:
    
    # STEP 1: Load the pre-trained Swiss model FIRST
    print("Loading pre-trained Swiss model...")
    model_filename = "nn_model_2025-07-14_CH_flexible.pt"
    
    # Define Swiss model architecture
    swiss_args = {
        'module': FlexibleNetwork,
        'nbFeatures': nInp,
        'module__input_dim': nInp,
        'module__dropout': 0.2,
        'module__hidden_layers': [128, 128, 64, 32],
        'module__use_batchnorm': True,
        'warm_start': True,             # CRITICAL: preserve pretrained weights
        'train_split': my_train_split,
        'batch_size': 128,
        'verbose': 1,
        'iterator_train__shuffle': True,
        'lr': 0.001,
        'max_epochs': 200,
        'optimizer': torch.optim.Adam,
        'optimizer__weight_decay': 1e-05,
        'callbacks': [
            ('early_stop', early_stop),
            ('lr_scheduler', lr_scheduler_cb),
            ('checkpoint', checkpoint_cb),
        ]
    }
    
    # Load the pre-trained model
    loaded_model = mbm.models.CustomNeuralNetRegressor.load_model(
        cfg, model_filename, **{**swiss_args, **param_init}
    )

    print("✓ Swiss model loaded successfully!")
    
    # STEP 2: Create datasets using Swiss model preprocessing
    print("Creating datasets with Swiss model...")
    features, metadata = loaded_model._create_features_metadata(df_X_train_subset)
    features_val, metadata_val = loaded_model._create_features_metadata(df_X_val_subset)
    
    # Create global datasets
    dataset = mbm.data_processing.AggregatedDataset(cfg,
                                                    features=features,
                                                    metadata=metadata,
                                                    targets=y_train)
    dataset = mbm.data_processing.SliceDatasetBinding(SliceDataset(dataset, idx=0),
                                                      SliceDataset(dataset, idx=1))
    
    dataset_val = mbm.data_processing.AggregatedDataset(cfg,
                                                        features=features_val,
                                                        metadata=metadata_val,
                                                        targets=y_val)
    dataset_val = mbm.data_processing.SliceDatasetBinding(SliceDataset(dataset_val, idx=0), 
                                                          SliceDataset(dataset_val, idx=1))
    
    print("train:", dataset.X.shape, dataset.y.shape)
    print("validation:", dataset_val.X.shape, dataset_val.y.shape)


    # STEP 3: Define progressive unfreezing strategy
    # Helper to freeze/unfreeze layers
    def set_requires_grad(layer_names, requires_grad=True):
        for name, param in loaded_model.module_.named_parameters():
            if name in layer_names:
                param.requires_grad = requires_grad

    # List of layer groups to progressively unfreeze
    layer_groups = [
        #(
            #[
                #'model.1.weight', 'model.1.bias',
                #'model.5.weight', 'model.5.bias',
                #'model.9.weight', 'model.9.bias',
                #'model.13.weight', 'model.13.bias'
            #],200,  0.1
        #),
        
        (['model.16.weight', 'model.16.bias'], 30, 0.01),
        (['model.12.weight', 'model.12.bias'], 20, 0.005),
        (['model.8.weight', 'model.8.bias'], 10, 0.001)
    ]

    # Start with all layers frozen
    print("Freezing all layers initially...")
    for name, param in loaded_model.module_.named_parameters():
        param.requires_grad = False

    # Progressive unfreezing loop
    for i, (layers, epochs, lr) in enumerate(layer_groups, 1):
        print(f"Stage {i}: Unfreezing {len(layers)//2} layer(s) for {epochs} epochs (lr={lr})...")
        
        # Unfreeze current layer group
        set_requires_grad(layers, True)
        
        # Update model parameters
        loaded_model = loaded_model.set_params(lr=lr, max_epochs=epochs)
        
        # Train current stage
        loaded_model.fit(features, y_train)
        
        # Evaluate current stage
        val_score = loaded_model.score(dataset_val.X, dataset_val.y)
        print(f"   Stage {i} validation score: {val_score:.4f}")
    
    # STEP 4: Save progressively fine-tuned model
    current_date = datetime.now().strftime("%Y-%m-%d")
    finetuned_model_filename = f"nn_model_progressive_{current_date}"
    loaded_model.save_model(finetuned_model_filename)
    print(f"Progressively fine-tuned model saved as: {finetuned_model_filename}")

else:
    print("Progressive training skipped (TRAIN=False)")



### Model Evaluation

#### Quick Performance Evaluation
Get immediate performance metrics on the test set using the fine-tuned model.

In [21]:
# Quick comprehensive evaluation of the fine-tuned model
print("Evaluating fine-tuned model performance...")

grouped_ids, scores_NN, ids_NN, y_pred_NN = evaluate_model_and_group_predictions(
    loaded_model, df_X_test_subset, test_set['y'], cfg, mbm
)

print("Test Set Performance Metrics:")
display(scores_NN)

# Validation score for confirmation that model with the best val_loss is used
val_score = loaded_model.score(dataset_val.X, dataset_val.y)
print(f"Validation score (for reference): {val_score:.4f}")

# Calculate additional performance metrics by glacier
print("\nPerformance by glacier:")
glacier_performance = grouped_ids.groupby('GLACIER').apply(
    lambda x: pd.Series({
        'n_samples': len(x),
        'rmse': np.sqrt(np.mean((x['target'] - x['pred'])**2)),
        'mae': np.mean(np.abs(x['target'] - x['pred'])),
        'r2': 1 - np.sum((x['target'] - x['pred'])**2) / np.sum((x['target'] - x['target'].mean())**2)
    })
).round(4)

display(glacier_performance.sort_values('rmse'))

Evaluating fine-tuned model performance...
Test Set Performance Metrics:


{'score': -1.1353379330383258,
 'mse': 1.1353379428498442,
 'rmse': 1.0655223802669958,
 'mae': 0.7522991885225665,
 'pearson': 0.9142183986990992}

Validation score (for reference): -1.1622

Performance by glacier:


,n_samples,rmse,mae,r2
GLACIER,,,,
RGI60-06.00479,1.0,0.3039,0.3039,-inf
Dyngjujoekull,240.0,0.6262,0.4483,0.8818
Eyjabakkajoekull,222.0,0.6722,0.5142,0.9240
RGI60-06.00409,6.0,0.6763,0.6279,-0.9695
RGI60-06.00465,79.0,0.6947,0.5204,-0.2006
RGI60-06.00232,43.0,0.6982,0.5292,0.8860
RGI60-06.00478,101.0,0.8349,0.6013,0.4870
Langjoekull Ice Cap,285.0,0.8402,0.6688,0.7879
Koeldukvislarjoekull,309.0,0.8642,0.6288,0.8117


#### Epoch-wise Model Evaluation
Evaluate models saved at different training epochs to understand training dynamics and optimal stopping points.

In [ ]:
# Evaluate models saved at different epochs to analyze training dynamics
print("Evaluating models saved at different training epochs...")

epochs_to_evaluate = [10, 15, 20, 30, 50, 100]
model_prefix = "nn_model_best_epoch"

epoch_results = {}

for epoch in epochs_to_evaluate:
    model_name = f"{model_prefix}_{epoch}.pt"
    
    # Check if model file exists
    if not os.path.exists(model_name):
        print(f"Model for epoch {epoch} not found, skipping...")
        continue

    print(f"Evaluating model at epoch {epoch}...")
    
    # Load model with same architecture as Swiss model
    epoch_model = mbm.models.CustomNeuralNetRegressor(
        cfg, **swiss_args, **param_init
    )
    epoch_model = epoch_model.set_params(device='cpu').to('cpu')
    epoch_model.initialize()
    
    # Load saved weights
    state_dict = torch.load(model_name, map_location='cpu')
    epoch_model.module_.load_state_dict(state_dict)

    # Evaluate the model
    grouped_ids_epoch, scores_NN_epoch, ids_NN_epoch, y_pred_NN_epoch = evaluate_model_and_group_predictions(
        epoch_model, df_X_test_subset, test_set['y'], cfg, mbm
    )
    
    # Store results
    epoch_results[epoch] = scores_NN_epoch
    
    print(f"Epoch {epoch} performance:")
    display(scores_NN_epoch)
    print("-" * 50)

print("Epoch-wise evaluation completed!")

# Compare performance across epochs
if epoch_results:
    print("\nPerformance comparison across epochs:")
    comparison_df = pd.DataFrame(epoch_results).T
    comparison_df.index.name = 'Epoch'
    display(comparison_df.round(4))

Evaluating models saved at different training epochs...
Evaluating model at epoch 10...
Epoch 10 performance:


{'score': -1.4062722418195912,
 'mse': 1.406272245011415,
 'rmse': 1.1858635018464034,
 'mae': 0.8299792948443621,
 'pearson': 0.9080360718256053}

--------------------------------------------------
Evaluating model at epoch 15...
Epoch 15 performance:


{'score': -1.4122355176131365,
 'mse': 1.4122355174869037,
 'rmse': 1.1883751585618507,
 'mae': 0.8056200029053975,
 'pearson': 0.9117571720726377}

--------------------------------------------------
Evaluating model at epoch 20...
Epoch 20 performance:


...

{'score': -1.2567828676256818,
 'mse': 1.2567828863550845,
 'rmse': 1.1210632838315082,
 'mae': 0.7633953157995983,
 'pearson': 0.9196548044204494}

--------------------------------------------------
Evaluating model at epoch 50...
Epoch 50 performance:


{'score': -1.2293091799955342,
 'mse': 1.2293091718595768,
 'rmse': 1.10874215751886,
 'mae': 0.7577432498879246,
 'pearson': 0.9215459395388584}

--------------------------------------------------
Model for epoch 100 not found, skipping...
Epoch-wise evaluation completed!

Performance comparison across epochs:


,score,mse,rmse,mae,pearson
Epoch,,,,,
10,-1.4063,1.4063,1.1859,0.8300,0.9080
15,-1.4122,1.4122,1.1884,0.8056,0.9118
20,-1.3612,1.3612,1.1667,0.7990,0.9130
30,-1.2568,1.2568,1.1211,0.7634,0.9197
50,-1.2293,1.2293,1.1087,0.7577,0.9215


#### Comprehensive Visualization and Analysis
Generate detailed visualizations to understand model performance across different glaciers and time periods.

In [ ]:
# Prepare comprehensive visualization data
print("Preparing data for comprehensive visualizations...")

# Create features and metadata for final evaluation
features_test, metadata_test = loaded_model._create_features_metadata(df_X_test_subset)

# Ensure all tensors are on CPU for visualization
if hasattr(features_test, 'cpu'):
    features_test = features_test.cpu()
if hasattr(test_set['y'], 'cpu'):
    targets_test = test_set['y'].cpu()
else:
    targets_test = test_set['y']

# Create final test dataset
dataset_test = mbm.data_processing.AggregatedDataset(cfg,
                                                     features=features_test,
                                                     metadata=metadata_test,
                                                     targets=targets_test)

dataset_test = [
    SliceDataset(dataset_test, idx=0),  # Features
    SliceDataset(dataset_test, idx=1)   # Targets
]

# Generate final predictions
print("Generating final predictions for visualization...")
y_pred = loaded_model.predict(dataset_test[0])
y_pred_agg = loaded_model.aggrPredict(dataset_test[0])

# Prepare evaluation metrics
batchIndex = np.arange(len(y_pred_agg))
y_true = np.array([e for e in dataset_test[1][batchIndex]])

# Calculate comprehensive performance metrics
score = loaded_model.score(dataset_test[0], dataset_test[1])
mse, rmse, mae, pearson = loaded_model.evalMetrics(y_pred, y_true)

print(f"Final Model Performance Summary:")
print(f"   R² Score: {score:.4f}")
print(f"   RMSE: {rmse:.4f} mm w.e.")
print(f"   MAE: {mae:.4f} mm w.e.")
print(f"   Pearson r: {pearson:.4f}")

# Create comprehensive results DataFrame
id = dataset_test[0].dataset.indexToId(batchIndex)
grouped_ids = pd.DataFrame({
    'target': [e[0] for e in dataset_test[1]],
    'ID': id,
    'pred': y_pred_agg
})

# Add comprehensive metadata
periods_per_ids = df_X_test_subset.groupby('ID')['PERIOD'].first()
grouped_ids = grouped_ids.merge(periods_per_ids, on='ID')

glacier_per_ids = df_X_test_subset.groupby('ID')['GLACIER'].first()
grouped_ids = grouped_ids.merge(glacier_per_ids, on='ID')

years_per_ids = df_X_test_subset.groupby('ID')['YEAR'].first()
grouped_ids = grouped_ids.merge(years_per_ids, on='ID')

print("Visualization data prepared successfully!")

Validation score (higher is better): -0.33718592127632885


In [ ]:
# Generate comprehensive publication-ready visualizations
print("Creating comprehensive visualizations...")

# 1. Time series predictions by glacier
print("   Generating time series predictions by glacier...")
PlotPredictions_NN(grouped_ids)

# 2. Predicted vs. observed scatter plot with performance metrics
print("   Creating prediction vs. truth scatter plot...")
predVSTruth_all(grouped_ids, mae, rmse, title='Transfer Learning: Swiss→Norwegian Glaciers')

# 3. Individual glacier performance analysis
print("   Generating individual glacier performance analysis...")
PlotIndividualGlacierPredVsTruth(grouped_ids, base_figsize=(20, 15))

print("All visualizations generated successfully!")

# Summary statistics by glacier for detailed analysis
print("\nDetailed Performance Summary by Glacier:")
glacier_stats = grouped_ids.groupby('GLACIER').agg({
    'target': ['count', 'mean', 'std'],
    'pred': ['mean', 'std']
}).round(4)

# Calculate RMSE and MAE per glacier
glacier_rmse = grouped_ids.groupby('GLACIER').apply(
    lambda x: np.sqrt(np.mean((x['target'] - x['pred'])**2))
).round(4)

glacier_mae = grouped_ids.groupby('GLACIER').apply(
    lambda x: np.mean(np.abs(x['target'] - x['pred']))
).round(4)

glacier_r2 = grouped_ids.groupby('GLACIER').apply(
    lambda x: 1 - np.sum((x['target'] - x['pred'])**2) / np.sum((x['target'] - x['target'].mean())**2)
).round(4)

# Combine all metrics
performance_summary = pd.DataFrame({
    'N_samples': glacier_stats[('target', 'count')],
    'RMSE': glacier_rmse,
    'MAE': glacier_mae,
    'R²': glacier_r2,
    'Target_mean': glacier_stats[('target', 'mean')],
    'Target_std': glacier_stats[('target', 'std')]
}).sort_values('RMSE')

print("Performance by glacier (sorted by RMSE):")
display(performance_summary)